In [ ]:
import pandas as pd
import re

In [26]:
temp = list()
with open('./gangmuk-locust-test/checkoutcart-W100,300,600-E100,100,100-C100,100,100-S100,100,100/bg0/SLATE-with-jumping-global-5/body2.csv', 'r') as f:
    section = 0
    for line in f:
        result = re.split(r'[ ,;:|]+', line)
        if result[0] == "asdf":
            section += 1
            continue
        # print(result)
        if len(result) == 13:
            result.insert(6, ' ')
        if len(result) == 14:
            result.append(section)
            temp.append(result)
            # for i in range(0, len(result)):
            #     print(f"[{i}]: {result[i]}")
            # break
            
# us-central-1 sslateingress-us-central-1 POST /cart/checkout 53af6da4cd4cf76ef8eb43b386e5ae71 f8eb43b386e5ae71  1731561898860 1731561898885 0 POST@/cart/checkout,3,0|

In [ ]:
df = pd.DataFrame(temp, columns=['region', 'service', 'method', 'url', 'trace_id', 'span_id', 'parent_span_id', 'st', 'et', 'callsize', 'ep', 'rps', 'inflight', "dummy", "section"])
df['st'] = df['st'].astype(int)
df['et'] = df['et'].astype(int)
df['callsize'] = df['callsize'].astype(int)
df['rps'] = df['rps'].astype(int)

df.drop(columns=['dummy'], inplace=True)
df = df[df['rps'] > 200]
avg_num_svc_in_trace = df.groupby('trace_id').size().mean()
df['num_span'] = df.groupby('trace_id')['span_id'].transform('count')
span_counts = df.groupby('trace_id')['span_id'].count()
trace_ids_with_complete_spans = span_counts[span_counts == 8].index
df_complete_traces = df[df['trace_id'].isin(trace_ids_with_complete_spans)]

trace_ids_with_incomplete_spans = span_counts[span_counts != 8].index
df_incomplete_traces = df[df['trace_id'].isin(trace_ids_with_incomplete_spans)]
df_incomplete_traces = df_incomplete_traces.sort_values(by='trace_id')
df_incomplete_traces.reset_index(drop=True, inplace=True)
total_num_traces = df['trace_id'].nunique()
num_incomplete_traces = df_incomplete_traces['trace_id'].nunique()

trace_ids_with_multiple_sections = df.groupby('trace_id')['section'].nunique()
trace_ids_with_multiple_sections = trace_ids_with_multiple_sections[trace_ids_with_multiple_sections > 1].index
cross_section_traces = df[df['trace_id'].isin(trace_ids_with_multiple_sections)]
num_cross_section_traces = cross_section_traces['trace_id'].nunique()

print(f"avg_num_svc_in_trace: {avg_num_svc_in_trace}")
print(f"#total traces: {total_num_traces}")
print(f"#incomplete traces: {num_incomplete_traces}")
print(f"#cross-section traces: {num_cross_section_traces}")

print(f"Ratio of incomplete traces: {int(((num_incomplete_traces+num_cross_section_traces) / total_num_traces)*100)}%")

# display(df_incomplete_traces)
display(cross_section_traces)
# cross_section_traces.to_csv('incomplete_traces.csv', index=False)
# df_incomplete_traces.to_csv('incomplete_traces.csv', index=False)

# Add a column that counts the number of unique regions per trace_id
region_count = df.groupby('trace_id')['region'].nunique()
df = df.merge(region_count.rename('num_regions'), on='trace_id')

# Filter for traces that involve more than one region
multi_region_traces = df[df['num_regions'] > 1]['trace_id'].unique()

# Display the first multi-region trace
if multi_region_traces.size > 0:
    first_trace_id = multi_region_traces[0]
    trace = df[df['trace_id'] == first_trace_id]
    print(f"traceid: {first_trace_id}")
    print(trace)
else:
    print("No multi-region traces found.")

avg_num_svc_in_trace: 7.645812904491077
#total traces: 5099
#incomplete traces: 575
#cross-section traces: 269
Ratio of incomplete traces: 16%


,region,service,method,url,trace_id,span_id,parent_span_id,st,et,rt,ep,rps,inflight,section,num_span,span_count
46911,us-west-1,paymentservice-us-west-1,POST,/hipstershop.PaymentService/Charge,85465ca2ff3bfca29bc34254c180050c,44a71287bfe450ae,6f811a7082159cfa,1731621759995,1731621759997,0,POST@/hipstershop.PaymentService/Charge,225,0,8,7,7
46914,us-west-1,paymentservice-us-west-1,POST,/hipstershop.PaymentService/Charge,ae9a568dd05f81690e28dba6f04128df,b488813cb9d196dc,aa7eb0241167954b,1731621760008,1731621760013,0,POST@/hipstershop.PaymentService/Charge,227,0,8,6,6
47954,us-central-1,paymentservice-us-central-1,POST,/hipstershop.PaymentService/Charge,6c4a85df526586d0b0b3516100039d66,a2961698de1dbad8,d0618420ad8432f8,1731621760004,1731621760009,0,POST@/hipstershop.PaymentService/Charge,236,0,8,8,8
48492,us-central-1,checkoutservice-us-central-1,POST,/hipstershop.CheckoutService/PlaceOrder,0dab2e04142b89db306eb5b92effa38e,33cff8d6363505ec,a1518eac5717cce6,1731621759952,1731621760063,0,POST@/hipstershop.CheckoutService/PlaceOrder,219,0,8,8,8
48857,us-east-1,cartservice-us-east-1,POST,/hipstershop.CartService/GetCart,b4a1324cb63d817be7600f8d33953b10,7a3dfdcc9cfbaf63,0bc834e32b07540d,1731621760029,1731621760034,0,POST@/hipstershop.CartService/GetCart,226,0,8,8,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92693,us-central-1,emailservice-us-central-1,POST,/hipstershop.EmailService/SendOrderConfirmation,ebd175d27dd056e519f101e85f249db2,93837b881eaad7ef,c26deba1e3ca11b3,1731621760958,1731621760960,0,POST@/hipstershop.EmailService/SendOrderConfir...,236,0,9,8,8
92890,us-south-1,frontend-us-south-1,POST,/cart/checkout,316fc736aec116a0bb46bd8b8f81cff8,13794a0a3f14d973,bb46bd8b8f81cff8,1731621760731,1731621760760,0,POST@/cart/checkout,238,0,9,8,8
92893,us-south-1,frontend-us-south-1,POST,/cart/checkout,81cf0a15072c24764e08bd123c80e8fa,b2b54adb17fa8e58,4e08bd123c80e8fa,1731621760944,1731621761078,0,POST@/cart/checkout,239,0,9,7,7
93318,us-south-1,paymentservice-us-south-1,POST,/hipstershop.PaymentService/Charge,203bb4922abbdc74c6071984e969f104,c2bbf1549a447fe8,90a873b5f8f2b0e1,1731621760985,1731621760986,0,POST@/hipstershop.PaymentService/Charge,240,0,9,7,7


No multi-region traces found.
